In [1]:
# 나만의 기상청 만들기
#데이터에 대한 설명 대전 지역에서 측정한 실내외 19곳의 센서데이터와, 주변 지역의 기상청 공공데이터를 semi-비식별화하여 제공
#센서는 온도를 측정하였다.
# 모든 데이터는 시간 순으로 정렬되어있다.
# 예측대상(target ariable)은 Y18이다.
"""
train.csv
30일간의 기상청 데이터(x00~x39) 및 센서데이터(y00~Y17)
이후 3일 간의 기상청 데이터(X00~X39) 및 센서 데이터(Y18)

test.csv
train.csv 기간 이후 80일 간의 기상청 데이터(x00~x39)
평가 방법은 mse로 평가한다. 
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 파일을 불러온다.
data = pd.read_csv("CSV_File/train.csv")

In [2]:
data
# 기온 : X00,X07,X28,X31,X32
# 현지기압 : X01,X06,X22,X27,X29
# 풍속 : X02, X03,X18,X24,X26
# 일일 누적 감수량: X04, X10,X21,X36,X39
# 해면 기압 X05,X08,X09,X23,X33
# 일일 누적 일사량 : X11,X14,X16,X19,X34
# 습도 : X12,X20,X30,X37,X38
#풍향 : X13,X15,X17,X25,X35

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4747,4747,19.9,987.6,0.9,0.8,0.0,1006.9,987.7,21.7,1007.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4748,4748,19.9,987.6,0.5,0.7,0.0,1006.8,987.7,21.6,1007.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4749,4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5


In [8]:
target = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08','Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18']
data[target].tail()

,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
4747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0


In [15]:
temperature = ['X00','X07','X28','X31','X32']# 기온 : X00,X07,X28,X31,X32
atm = ['X01','X06','X22','X27','X29']# 현지기압 : X01,X06,X22,X27,X29 atmospheric[air barometric] pressure
wind = ['X02','X03','X18','X24','X26']# 풍속 : X02, X03,X18,X24,X26
rain = ['X04','X10','X21','X36','X39']# 일일 누적 감수량: X04, X10,X21,X36,X39
sea = ['X05','X08','X09','X23','X33']# 해면 기압 X05,X08,X09,X23,X33
solar = ['X11','X14','X16','X19','X34']# 일일 누적 일사량 : X11,X14,X16,X19,X34
hum = ['X12','X20','X30','X37','X38']# 습도 : X12,X20,X30,X37,X38 humidity
wind_direction = ['X13','X15','X17','X25','X35'] #풍향 : X13,X15,X17,X25,X35
data[hum].head()

,X12,X20,X30,X37,X38
0,82.4,91.0,69.1,77.2,62.6
1,81.2,91.9,70.3,77.3,63.5
2,86.1,93.3,71.5,77.3,63.9
3,87.7,93.4,73.2,77.5,64.5
4,88.9,94.1,74.3,78.0,65.0


In [9]:
target = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08','Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18']
data[target].head()

,Y00,Y01,Y02,Y03,Y04,Y05,Y06,Y07,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,12.5,11.5,11.0,20.0,20.0,10.5,10.0,10.5,10.0,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,12.5,11.0,11.0,20.0,20.0,10.0,9.5,10.0,9.5,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,12.0,11.0,11.0,19.5,19.5,10.0,9.0,10.0,9.0,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,12.0,11.0,11.0,19.5,19.5,9.5,9.0,9.5,9.0,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,12.0,10.5,10.5,19.0,19.5,9.5,8.5,9.5,8.5,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN
